<a href="https://colab.research.google.com/github/pikey-msc/RiesgosFinancieros/blob/master/2020-1/Insumos/MertonKMV_Rb_P.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Paso de carga de librerías en R de todas las que se utilizarán

In [1]:
install.packages("quantmod")
install.packages("curl")
install.packages("data.table")
install.packages("RCurl")
install.packages("bitops")
install.packages("setInternet2")
install.packages("rootSolve")
install.packages("xml2")
library(rootSolve)
library(quantmod)
library(RCurl)
library(data.table)
library(xml2)
require(jsonlite) #para el getQuotes


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘xts’, ‘zoo’, ‘TTR’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘bitops’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“package ‘setInternet2’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘li

In [2]:
Symbols<-c  ("WALMEX.MX")#tienen que ir en orden alfabético
length(Symbols)


start_date=Sys.Date()-3660 #fecha inicial

#Creación del objeto para guardar los datos
dataEnv<-new.env()

#obtener los datos
getSymbols.yahoo(Symbols,env=dataEnv,from=start_date)


#limpiarlos, alinearnos y quedarnos con el precio de cierre 



#muestra de datos
#Nos quedamos con los precios
stock_prices = na.omit(dataEnv$`WALMEX.MX`)
stock_prices=stock_prices[,4] #Close
tail(stock_prices[,])


[1] 1

Warning message:
“WALMEX.MX contains missing values. Some functions will not work if objects contain missing values in the middle of the series. Consider using na.omit(), na.approx(), na.fill(), etc to remove or replace them.”


[1] "WALMEX.MX"

           WALMEX.MX.Close
2021-01-07           58.71
2021-01-08           60.75
2021-01-11           60.92
2021-01-12           61.75
2021-01-13           63.19
2021-01-14           64.51

Se deja el código de "webscrapping" hay que arreglarlo

In [29]:

#PARA CARGAR EL NÚMERO DE ACCIONES EN VENTA

getQuote <- function(ticks) {
  qRoot <- "https://query1.finance.yahoo.com/v7/finance/quote?fields=symbol,longName,regularMarketPrice,regularMarketChange,regularMarketTime,sharesOutstanding,bookValue&formatted=false&symbols="
  z <- fromJSON(paste(qRoot, paste(ticks, collapse=","), sep=""))
  z <- z$quoteResponse$result[,c("symbol", "regularMarketTime", "regularMarketPrice", "regularMarketChange", "longName", "sharesOutstanding", "bookValue" )]
  row.names(z) <- z$symbol
  z$symbol <- NULL
  names(z) <- c("Time", "Price", "Change", "Name", "Numeroacciones", "valorlibros")
  z$Time <- as.POSIXct(z$Time, origin = '1970-01-01 00:00:00')
  return(z)
}


ticks=Symbols

stqt=getQuote(Symbols)


#PARA CARGAR LOS ESTADOS FINANCIEROS Y FLUJOS DE EFECTIVO
url <- "https://finance.yahoo.com/quote/"
stock=Symbols
# scrapy_stocks <- function(stock){
  if ("rvest" %in% installed.packages()) {
    library(rvest)
  }else{
    install.packages("rvest")
    library(rvest)
  }
  for (i in 1:length(stock)) {
    tryCatch(
      {
        # Income Statement
        financial <- paste0(url,stock[i],"/financials?p=",stock[i])
        wahis.session <- html_session(financial)                                
        p <-    wahis.session %>% 
          html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[4]/div[1]/div[1]/div[2]') %>%
          .[[1]] %>% html_children() %>% html_nodes("div")
        suppressWarnings(Values <- p[sapply(p, function(x){length(x %>% html_attrs())}) == 3] %>% 
                           html_text() %>% gsub(pattern = ",",replacement = "") %>% 
                           gsub(pattern = "-",replacement = 0) %>% as.numeric()) 
        Headers <- wahis.session %>% 
          html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[4]/div[1]/div[1]') %>% 
          .[[1]] %>% html_nodes("span") %>% html_text() %>% head()
        Concepts <- p[sapply(p, function(x){length(x %>% html_attrs())}) == 2] %>% html_text() %>% .[seq(2,88,4)]
        print(Concepts)
        # print(data.frame(Breakdown = Concepts[-c(3,6,9,12,15,18,21)], matrix(Values[!is.na(Values)], ncol = 1, byrow = T)))
        print(data.frame(Breakdown = Concepts[-c(2,5,8,11,14,17,20)], matrix(Values[!is.na(Values)], ncol = 1, byrow = T)))
        IS <- data.frame(Breakdown = Concepts[-c(4,16,18)], matrix(Values[!is.na(Values)], ncol = 5, byrow = T))
        names(IS) <- Headers
        temp1 <- IS
        
        # Balance Sheet
        bsheet <- paste0(url,stock[i],"/balance-sheet?p=",stock[i])
        wahis.session <- html_session(bsheet)
        p <-    wahis.session %>% 
          html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[4]/div[1]/div[1]/div[2]') %>%
          .[[1]] %>% html_children() %>% html_nodes("div")
        suppressWarnings(Values <- p[sapply(p, function(x){length(x %>% html_attrs())}) == 3] %>% 
                           html_text() %>% gsub(pattern = ",",replacement = "") %>% gsub(pattern = "-",replacement = 0) %>% as.numeric()) 
        Headers <- wahis.session %>% 
          html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[4]/div[1]/div[1]') %>% 
          .[[1]] %>% html_nodes("span") %>% html_text() %>% head(5)
        Concepts <- p[sapply(p, function(x){length(x %>% html_attrs())}) == 2] %>% html_text() %>% .[seq(2,175,4)]
        print(Concepts[10][1:2])
        BS <- data.frame(Breakdown = Concepts[-c(1:3,11,12,22:24,32,39)], matrix(Values[!is.na(Values)], ncol = 4, byrow = T))
        print(data.frame(Breakdown = Concepts[c(6)], matrix(Values[!is.na(Values)], ncol = 4, byrow = T)))
        names(BS) <- Headers
        temp2 <- BS
        
        # Cash flows
        cash_f <- paste0(url,stock[i],"/cash-flow?p=",stock[i])
        wahis.session <- html_session(cash_f)
        p <-    wahis.session %>% 
          html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[4]/div[1]/div[1]/div[2]') %>%
          .[[1]] %>% html_children() %>% html_nodes("div")
        suppressWarnings(Values <- p[sapply(p, function(x){length(x %>% html_attrs())}) == 3] %>% 
                           html_text() %>% gsub(pattern = ",",replacement = "") %>% gsub(pattern = "-",replacement = 0) %>% as.numeric())
        Headers <- wahis.session %>% 
          html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[4]/div[1]/div[1]') %>% 
          .[[1]] %>% html_nodes("span") %>% html_text() %>% head()
        Concepts <- p[sapply(p, function(x){length(x %>% html_attrs())}) == 2] %>% html_text() %>% .[seq(2,133,4)]
        CF <- data.frame(Breakdown = Concepts[-c(1,13,20,30)], matrix(Values[!is.na(Values)], ncol = 5, byrow = T))
        names(CF) <- Headers
        temp3 <- CF
        
        # Shares Outstanding
        stock_url <- paste0(url,stock[i],"/key-statistics?p=",stock[i])
        SO <- html_session(stock_url) %>%
          html_nodes(xpath = '//*[@id="Col1-0-KeyStatistics-Proxy"]/section/div[2]/div[2]/div/div[2]/div/table/tbody/tr[3]/td[2]') %>% 
          html_text() %>% gsub(pattern = "[A-z]",replacement = "") %>% as.numeric()
        temp4 <- SO
        
        assign(paste0(stock[i],'.f'),value = list(IS = temp1,BS = temp2,CF = temp3, SO = temp4),envir = parent.frame())
        
      },
      error = function(cond){
        message(stock[i], "Give error ",cond)
      }
    )
  }
# }

#stock=Symbols
scrapy_stocks(Symbols) 
#attach(WALMEX.MX.f)
#WALMEX.MX.f$BS
#WALMEX.MX.f$IS
#WALMEX.MX.f$CF
BS


 [1] "Total Revenue"                                                  
 [2] ""                                                               
 [3] "Operating Expense103,038,16494,048,11792,396,72483,857,809"     
 [4] "Operating Income"                                               
 [5] ""                                                               
 [6] "Tax Provision15,526,38511,304,34712,107,12310,899,663"          
 [7] "Net Income Common Stockholders"                                 
 [8] ""                                                               
 [9] "Diluted EPS-0.00220.00210.0023"                                 
[10] "Basic Average Shares"                                           
[11] ""                                                               
[12] "Rent Expense Supplemental-6,775,941--"                          
[13] "Total Expenses"                                                 
[14] ""                                                               
[15] "

WALMEX.MXGive error Error in data.frame(Breakdown = Concepts[-c(2, 5, 8, 11, 14, 17, 20)], : arguments imply differing number of rows: 15, 128


Warning message in matrix(Values[!is.na(Values)], ncol = 5, byrow = T):
“data length [128] is not a sub-multiple or multiple of the number of rows [26]”
WALMEX.MXGive error Error in data.frame(Breakdown = Concepts[-c(4, 16, 19)], matrix(Values[!is.na(Values)], : arguments imply differing number of rows: 19, 26




ERROR: ignored

In [3]:
TSE <- "Total stockholders' equity" #Capital
TL <- "Total Liabilities" #Deuda Actual
#dt <- colnames(BS)[2] # ÚLTIMA FECHA DISPONIBLE
dt="12/31/2019"



#K_0=BS[33,1]/1000 #Total Stockholder Equity VALOR DE LA EMPRESA
#en millones https://finance.yahoo.com/quote/WALMEX.MX/balance-sheet?p=WALMEX.MX
#Total stockholders' equity
#En millones
#K_0=BS[BS$Breakdown==TSE, dt]*1000/1000000
K_0=168144767*1000/1000000
#nrow(BS)
#D_0=BS[24,1]/1000 #Total Liabilities DEUDA ACTUAL
#En millones
#D_0=BS[BS$Breakdown==TL, dt]*1000/1000000
D_0=181027502*1000/1000000


#Shares Outstanding (Se expresa en miles de millones en la página)
#En millones
#Numeroacciones=WALMEX.MX.f$SO*1000000000/1000000
Numeroacciones=17.46*1000000000/1000000

dt=strptime(dt, "%m/%d/%Y")
dt=format(dt, "%Y-%m")
dt
#no esta sentencia entonces mi ultima fecha fue tanto 
n=nrow(stock_prices[paste("/",dt,sep="")]) #NUMERO DE FILAS A ANALIZAR PARA EL ÚLTIMO DATO

A_0=stock_prices[paste("/",dt,sep="")][n] #valores actuales

#A_0=A_0*stqt$Numeroacciones/1000000 #VALOR ACTUAL DE LAS ACCIONES VALOR DE ACCIÓN POR ACCIONES DISPONIBLES
A_0=A_0*Numeroacciones #VALOR ACTUAL DE LAS ACCIONES VALOR DE ACCIÓN POR ACCIONES DISPONIBLES
A_0/K_0

A=stock_prices[paste("/",dt,sep="")]  #VECTOR DE PRECIOS 

vol_A=sqrt(var(log(as.matrix(A[2:n])/as.matrix(A[1:(n-1)]))))*sqrt(252) #VOLATILIDAD ANUAL

#Valor del CETE
r=0.0425
T=1

A_0
K_0
vol_A
D_0
r
T


[1] "2019-12"

           WALMEX.MX.Close
2019-12-31        5.626002

           WALMEX.MX.Close
2019-12-31        945982.8

[1] 168144.8

,WALMEX.MX.Close
WALMEX.MX.Close,0.2407236


[1] 181027.5

[1] 0.0425

[1] 1

In [7]:
x=c(D_0,vol_A)

#Función para estimar vol_k y D
G = function(x) 
{
  vol_k <- x[2]
  D <- x[1]
  c(A_0-K_0*pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))+D*exp(-r*T)*pnorm((log(K_0/D)+(r-0.5*vol_k^2)*T)/(vol_k*sqrt(T))),
  A_0*vol_A-vol_k*K_0*pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T))) )
}

#K_0=164370

ss=multiroot(G,x,positive = TRUE) #RESOLVER EL SISTEMA.

D=ss$root[1]
vol_k=ss$root[2]
print("Cociente de D/D_0")
ss$root[1]/D_0
print("Cociente de vol_A/vol_K)")
vol_A/ss$root[2]
print("Función G")
G(c(D,vol_k))

#Probabilidad de incumplimiento
PD_merton=1-pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))
PD_merton


Warning message in stode(y, times, func, parms = parms, ...):
“steady-state not reached”


[1] "Cociente de D/D_0"


[1] 0

[1] "Cociente de vol_A/vol_K)"


,WALMEX.MX.Close
WALMEX.MX.Close,0.1777461


[1] "Función G"


           WALMEX.MX.Close
2019-12-31          777838
2019-12-31               0

[1] 0

In [10]:
#Sólo estimar la volatilidad dada la deuda
#Probabilidad de Incumplimiento

Vk = function(x) 
{
  vol_k <- x
  ((vol_k/vol_A)*(K_0/A_0))-pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))
}

#Definir una Deuda
D=D_0*1/2
x=vol_A #semilla de la vol_k
ss=multiroot(Vk,x,positive = TRUE) #RESOLVER EL SISTEMA.

vol_k=ss$root
print("Ratio vol_k/vol_A")
ss$root[1]/vol_A
print("valor Vk bajo solución")
Vk(vol_k)

PD_merton=1-pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))

print("Deuda estimada")
D
print("Deuda sencilla")
D_0
print("Vol estimada")
vol_k
print("Vol semilla")
vol_A
print("PD Merton")
PD_merton



[1] "Ratio vol_k/vol_A"


,WALMEX.MX.Close
WALMEX.MX.Close,4.923513


[1] "valor Vk bajo solución"


           WALMEX.MX.Close
2019-12-31   -1.852787e-08

[1] "Deuda estimada"


...

[1] 1.185206

[1] "Vol semilla"


,WALMEX.MX.Close
WALMEX.MX.Close,0.2407236


[1] "PD Merton"


[1] 0.1248647

#KMV

In [11]:
print("A_0:")
A_0
print("Valor de K_0")
K_0
print("Valor de vol_A")
vol_A
print("Valor de Deuda Actual")
D_0
print("Tasa libre de riesgo")
r
print("Temporalidad")
T


#Función para estimar vol_k y K_0
Gkmv = function(x) 
{
  vol_k <- x[2]
  K_0 <- x[1]
  c(A_0-K_0*pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))+D*exp(-r*T)*pnorm((log(K_0/D)+(r-0.5*vol_k^2)*T)/(vol_k*sqrt(T))),
    A_0*vol_A-vol_k*K_0*pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T))) )
}

D=D_0

x=c(K_0,vol_A)

ss=multiroot(Gkmv,x,positive = TRUE) #RESOLVER EL SISTEMA.

K_e=ss$root[1]
vol_k=ss$root[2]
print("Cociente K_0/D_0")
K_e/D_0
print("Cociente vol_K/vol_A")
ss$root[2]/vol_A
print("Valor de solución en función:")
Gkmv(c(K_e,vol_k))

DI=(K_e-D)/(vol_k*K_0)
print("DI")
DI
EDF=pnorm(-DI)
print("EDF Normal")
EDF
 


[1] "A_0:"


           WALMEX.MX.Close
2019-12-31        945982.8

[1] "Valor de K_0"


[1] 168144.8

[1] "Valor de vol_A"


...

           WALMEX.MX.Close
2019-12-31   -2.910383e-11
2019-12-31   -2.910383e-11

[1] "DI"


[1] 27.43731

[1] "EDF Normal"


[1] 4.923453e-166